### 7.1使用\_\_slots\_\_
动态语言可以动态的给实例绑定属性和方法

In [3]:
from types import MethodType
class Student(object):
    pass
s = Student()
s.name = 'Michael' # 动态给实例绑定一个属性
print(s.name)

def set_age(self, age): # 定义一个函数作为实例方法
    self.age = age
s.set_age = MethodType(set_age, s) # 给实例绑定一个方法
s.set_age(25) # 调用实例方法
s.age # 测试结果   


Michael


25

**但对实例新绑定的方法对其他实例是不起作用的**  
为了使新绑定的方法和属性可以对所有实例都起效，可以对类绑定方法

In [5]:
def set_score(self, score):
    self.score = score

Student.set_score = set_score

通过定义特殊变量**\_\_slots\_\_**,限制可添加的属性名称，该属性仅对当前类有效，对其子类无效

In [7]:
class Student(object):
    __slots__ = ('name', 'age') # 用tuple定义允许绑定的属性名称
s = Student() # 创建新的实例
s.name = 'Michael' # 绑定属性'name'
s.age = 25 # 绑定属性'age'
s.score = 99 # 绑定属性'score'

AttributeError: 'Student' object has no attribute 'score'

### 7.2使用@property
使用=号为实例新增属性时，我们无法检查类型  
Python内置的@property装饰器就是负责**把一个方法变成属性调用的**

In [12]:
class Student(object):
    @property
    def score(self):
        return self._score
    @score.setter
    def score(self,value):
        if not isinstance(value,int):
            raise ValueError('Must be an integer!')
        if value<0 or value>100:
            raise ValueError('Must in 0~100!')
        self._score = value
        
s = Student()
s.score = 60
print(s.score)
#s.score = 9999

60


In [38]:
class Screen(object):
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self,value):
        if not isinstance(value,int):
            raise TypeError('Not a Integer')
        self._height = value
        
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self,value):
        if not isinstance(value,int):
            raise TypeError('Not a Integer')
        self._width = value
        
    @property
    def resolution(self):
        return self._height * self._width
        
s = Screen()
s.width = 1024
s.height = 768
print(s.width)
print(s.height)
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过!')
else:
    print('测试失败!')

1024
768
resolution = 786432
测试通过!


###  7.3多重继承
通过在括号中填入多种需要继承的类即可，如：  
class Dog(Mammal, Runnable):   
MRO的问题  
多继承时C3排序，从根开始寻找入度为0的节点，若同时有多个入度为0的点，则取最左，然后移除该节点，重新寻找入度为0的点  
http://python.jobbole.com/85685/  




**Mixin**  类似C#中的Interface，但是更简单  
Mixin 就是混入的意思。和多重继承类似（其实可以把 Mixin 看作多重继承的一种在特定场景下的应用），但通常混入 Mixin 的类和 Mixin 类本身不是 is-a 的关系，混入 Mixin 类是为了添加某些（可选的）功能。   
自由地混入 Mixin 类就可以灵活地为被混入的类添加不同的功能。传统的「接口」概念中并不包含实现，而 Mixin 包含实现

+ TagMixin 类是单一职责的TagMixin   
+ 类对宿主类（Post）一无所知，除了要求宿主类有 ident 和 kind 这两个属性（等价于 Java 中要求宿主类实现 Entity 接口）  
+ 宿主类的主体逻辑不会因为去掉 TagMixin 而受到影响，同时也不存在超类方法调用（super）以避免引入 MRO 查找顺序问题  

在Python中与多重继承没有任何区别，Mixin仅是换个名字，作为一种编程规范

###  7.4定制类
形如\_\_xx\_\_的变量名或函数名需注意，这些在Python中是有特殊用途的  
可以帮助我们定制类  
如想要print()自定类的效果时，可以使用\_\_str\_\_()方法


In [40]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name: %s)' % self.name
    
print(Student('Michael'))


Student object (name: Michael)


In [41]:
#当直接调用s时，使用的是__repr__()
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name=%s)' % self.name
    #偷懒写法，将__repr__方法直接指向__str__
    __repr__ = __str__

**\_\_iter\_\_**  
如果一个类想被用于for ... in循环，类似list或tuple那样，就必须实现一个\_\_iter\_\_()方法，该方法返回一个迭代对象，然后，Python的for循环就会不断调用该迭代对象的\_\_next\_\_()方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。

In [67]:
class fib(object):
    def __init__(self):
        self.a = 0
        self.b = 1
    def __iter__(self):
        return self
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b # 计算下一个值
        if self.a>10000:
            raise StopIteration()
        return self.a
b = fib()

**\_\_getitem\_\_**   
通过下标取出元素，切片，字典排序

In [83]:
#取数
class Fib(object):
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a + b
        return a

In [84]:
f = Fib()
f[10]

89

In [85]:
#切片
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int): # n是索引
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice): # n是切片
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a, b = b, a + b
            return L

In [87]:
f = Fib()
f[0:5]

[1, 1, 2, 3, 5]

**\_\_getattr\_\_**  
自定义：查找属性不存在时的返回值（或函数）

In [91]:
class Student(object):

    def __init__(self):
        self.name = 'Michael'

    def __getattr__(self, attr):
        if attr=='score':
            return 99

In [92]:
s = Student()
s.name
s.score

99

此外，注意到任意调用如s.abc都会返回None，这是因为我们定义的__getattr__默认返回就是None。要让class只响应特定的几个属性，我们就要按照约定，抛出AttributeError的错误

In [96]:

class Student(object):

    def __init__(self):
        self.name = 'Michael'

    def __getattr__(self, attr):
        if attr=='score':
            return 99
        if attr=='age':
            return lambda: 25
        raise AttributeError('\'Student\' object has no attribute \'%s\'' % attr)

s = Student()
print(s.name)
print(s.score)
print(s.age())
# AttributeError: 'Student' object has no attribute 'grade'
print(s.grade)

Michael
99
25


AttributeError: 'Student' object has no attribute 'grade'

**\_\_call\_\_**     
直接在实例本身上调用  
class abc()  
定义\_\_call\_\_后可以直接调用该实例
abc()

仿佛类是一个函数

In [97]:
class Student(object):
    def __init__(self, name):
        self.name = name

    def __call__(self):
        print('My name is %s.' % self.name)

s = Student('Michael')
s()

My name is Michael.


### 7.5枚举类
定义常量是，一是通过变量名大写来定义，缺点是类型是int，且仍为变量  
更好的方法为：为这样的枚举类型定义一个类，每个常量都是类的唯一实例，通过Enum来实现


In [107]:
from enum import Enum
Month = Enum('Month',('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

In [109]:
for name,member in Month.__members__.items():
    print(name,':',member,':',member.value)

Jan : Month.Jan : 1
Feb : Month.Feb : 2
Mar : Month.Mar : 3
Apr : Month.Apr : 4
May : Month.May : 5
Jun : Month.Jun : 6
Jul : Month.Jul : 7
Aug : Month.Aug : 8
Sep : Month.Sep : 9
Oct : Month.Oct : 10
Nov : Month.Nov : 11
Dec : Month.Dec : 12


如果想要更加精准的控制枚举类型对应的值，可以从enum派生出自定义的类|

In [127]:
from enum import Enum,unique
#帮助检查没有重复值
@unique
class Constant(Enum):
    PI=3.14159
    Sun = 0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6

In [136]:
day1 = Constant.Mon
print(day1)
print(Constant.Tue)
print(Constant['Tue'])
print(Constant.Tue.value)
Constant(6)

Constant.Mon
Constant.Tue
Constant.Tue
2


<Constant.Sat: 6>

In [139]:
class Gender(Enum):
    Male = 0
    Female = 1

class Student(object):
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender

In [140]:
bart = Student('Bart', Gender.Male)
if bart.gender == Gender.Male:
    print('测试通过!')
else:
    print('测试失败!')

测试通过!


###  7.6使用元类

In [147]:
def fn(self, name='world'): # 先定义函数
    print('Hello, %s.' % name)


    Hello = type('Hello', (object,), dict(hello=fn)) # 创建Hello class

+ type()函数可以查看一个类型或变量的类型，Hello是一个class，它的类型就是type，而h是一个实例，它的类型就是class Hello
+ type()函数既可以返回一个对象的类型，又可以创建出新的类型  
   
要创建一个class对象，type()函数依次传入3个参数：  

1. class的名称；  
2. 继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；  
3. class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上。  

无须事先定义好类，可以写一些函数，在程序执行的过程中，有需要的话可以将一些功能封装为一个类

**元类metaclass**   
元类可以理解为用于创建类对象的类  
内建的元类为type，也就是说无论是预先定义的class还是动态创建的class都是元类的一个对象   
因此我们是用type(类名)时，返回的类型为type  
我们也可以自己定义元类
**基本用不到，在某些应用场景会用到，例如用于批量修改输出类型**